In [1]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('./axial2.json')
project = 'axial-module-393809'

In [2]:
from google.cloud import storage

bucket_name = 'dlp-bucket-dev-axial'
storage_client = storage.Client(project = project, credentials = credentials)
bucket = storage_client.get_bucket(bucket_name)
blobs = bucket.list_blobs()
print(blobs)
for b in blobs:
    print(b)
print("hello")

<Blob: dlp-bucket-dev-axial, dlp_sample.xlsx, 1693994739910369>
hello


In [62]:
import google.cloud.dlp

In [4]:
from google.cloud import bigquery
client = bigquery.Client(project = "axial-module-393809", credentials=credentials)
storage_client = storage.Client(project = project, credentials = credentials)

dataset_id = "axial-module-393809.axial_dataset_sample"
tables = client.list_tables(dataset_id)

for table in tables:
    print(f"{table.project} {table.dataset_id} {table.table_id}")

axial-module-393809 axial_dataset_sample sample_data


In [ ]:
dlp = google.cloud.dlp_v2.DlpServiceClient(credentials = credentials)

project_id = "axial-module-393809"
location_id = "us-central1-c"
language_code = "en-IN"
parent = f"projects/{project_id}/languageCode/{language_code}"
response = dlp.list_info_types(
                request={"parent": parent}
            )
# response.info_types.categories
for infotype in response.info_types:
    if infotype.categories:
        for category in infotype.categories:
#             print(str(category.location_category.name))
            if category.location_category.name == "INDIA":
                print(infotype.name)
# print((response.info_types))
# for info_type in response.info_types:
#     print(len(info_type.name))
#     print(f"{info_type.name}: {info_type.display_name}")

In [3]:
import threading
from typing import List, Optional
import google.cloud.dlp
import google.cloud.pubsub as pubsub

project = "axial-module-393809"
dlp = google.cloud.dlp_v2.DlpServiceClient(credentials=credentials)

info_types = ["FIRST_NAME", "LAST_NAME", "EMAIL_ADDRESS", "PASSWORD"]
info_types = [{"name": info_type} for info_type in info_types]

inspect_config = {
    "info_types": info_types,
    "min_likelihood": "LIKELY",
}

storage_config = {
    "big_query_options": {
        "table_reference": {
            "project_id": project,
            "dataset_id": "axial_dataset_sample",
            "table_id": "sample_data",
        }
    }
}

parent = f"projects/{project}/locations/global"
topic_id = "inspection-topic"
topic = google.cloud.pubsub.PublisherClient.topic_path(project, topic_id)
actions = [{"pub_sub": {"topic": topic}}]

inspect_job = {
    "inspect_config": inspect_config,
    "storage_config": storage_config,
    "actions": actions,
}

operation = dlp.create_dlp_job(request={"parent": parent, "inspect_job": inspect_job})
print(f"Inspection operation started: {operation.name}")

subscription_id = "inspection-sub"
subscriber = google.cloud.pubsub.SubscriberClient()
subscription_path = subscriber.subscription_path(project, subscription_id)
job_done = threading.Event()

def callback(message: google.cloud.pubsub_v1.subscriber.message.Message) -> None:
    try:
        if message.attributes["DlpJobName"] == operation.name:
            message.ack()
            message_body = message.data.decode("utf-8")
            publisher = pubsub.PublisherClient()
            publisher.publish(topic, message_body.encode("utf-8"))

            job = dlp.get_dlp_job(request={"name": operation.name})
            print(f"Job name: {job.name}")
            if job.inspect_details.result.info_type_stats:
                for finding in job.inspect_details.result.info_type_stats:
                    print(
                        "Info type: {}; Count: {}".format(
                            finding.info_type.name, finding.count
                        )
                    )
            else:
                print("No findings.")

            job_done.set()
        else:
            message.drop()
    except Exception as e:
        print(e)
        raise

timeout = 10
subscriber.subscribe(subscription_path, callback=callback)
finished = job_done.wait(timeout=timeout)
if not finished:
    print(
        "No event received before the timeout. Please verify that the "
        "subscription provided is subscribed to the topic provided."
    )


Inspection operation started: projects/axial-module-393809/locations/global/dlpJobs/i-6341251296196257181
No event received before the timeout. Please verify that the subscription provided is subscribed to the topic provided.


Failed to publish 1 messages.
Traceback (most recent call last):
  File "C:\Users\vbukka3\OneDrive - DXC Production\Documents\dxc_github\opsMate\opsmate-env\lib\site-packages\google\api_core\grpc_helpers.py", line 65, in error_remapped_callable
    return callable_(*args, **kwargs)
  File "C:\Users\vbukka3\OneDrive - DXC Production\Documents\dxc_github\opsMate\opsmate-env\lib\site-packages\grpc\_channel.py", line 1161, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "C:\Users\vbukka3\OneDrive - DXC Production\Documents\dxc_github\opsMate\opsmate-env\lib\site-packages\grpc\_channel.py", line 1004, in _end_unary_response_blocking
    raise _InactiveRpcError(state)  # pytype: disable=not-instantiable
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "One or more messages in the publish request is empty. Each message must contain either non-empty data, or at least one attrib

Job name: projects/axial-module-393809/locations/global/dlpJobs/i-6341251296196257181
Info type: EMAIL_ADDRESS; Count: 3
Info type: PASSWORD; Count: 3


In [4]:

import threading
import google.cloud.dlp
from google.protobuf.json_format import MessageToJson
from google.cloud import bigquery
import time


project = "axial-module-393809"
findings_project_id = "axial-module-393809"
findings_dataset_id = "result_dataset"
findings_table_id = "result_table1"


# Initialize DLP client
dlp = google.cloud.dlp_v2.DlpServiceClient(credentials=credentials)
bigquery_client = bigquery.Client(project=project)

info_types = ["FIRST_NAME", "LAST_NAME", "EMAIL_ADDRESS", "PASSWORD"]
info_types = [{"name": info_type} for info_type in info_types]

inspect_config = {
    "info_types": info_types,
    "min_likelihood": "LIKELY",
}

storage_config = {
    "big_query_options": {
        "table_reference": {
            "project_id": project,
            "dataset_id": "axial_dataset_sample",
            "table_id": "sample_data",
        }
    }
}

output_config = {
    "table": {
        "project_id": project,
        "dataset_id": findings_dataset_id,
        "table_id": findings_table_id,
    },
    "output_schema": "BASIC_COLUMNS",
}

save_findings_action = {"save_findings": {"output_config": output_config}}

parent = f"projects/{project}/locations/global"

inspect_job = {
    "inspect_config": inspect_config,
    "storage_config": storage_config,
    "actions": [save_findings_action],
}

operation = dlp.create_dlp_job(request={"parent": parent, "inspect_job": inspect_job})
print(f"Inspection operation started: {operation.name}")

while True:
    job = dlp.get_dlp_job(request={"name": operation.name})
    if job.state == google.cloud.dlp_v2.DlpJob.JobState.DONE:
        break
    time.sleep(5) 

print("DLP inspection completed.")
print(job)

# if job.inspect_details and job.inspect_details.result:
#     result = job.inspect_details.result

#     processed_bytes = result.processed_bytes
#     total_estimated_bytes = result.total_estimated_bytes
#     info_type_stats = result.info_type_stats

#     print("Processed Bytes:", processed_bytes)
#     print("Total Estimated Bytes:", total_estimated_bytes)

#     for info_type_stat in info_type_stats:
#         info_type_name = info_type_stat.info_type.name
#         count = info_type_stat.count
#         print("Info Type:", info_type_name)
#         print("Count:", count)
# else:
#     print("No DLP findings found.")


# schema = [
#     bigquery.SchemaField("Processed_Bytes", "INTEGER"),
#     bigquery.SchemaField("Total_Estimated_Bytes", "INTEGER"),
#     bigquery.SchemaField("Info_Type", "STRING"),
#     bigquery.SchemaField("Count", "INTEGER"),
# ]

# dataset_ref = bigquery_client.dataset(bq_dataset)
# table_ref = dataset_ref.table(bq_table)

# try:
#     table = bigquery.Table(table_ref, schema=schema)
#     table = bigquery_client.create_table(table)
#     print(f"Table {project}.{bq_dataset}.{bq_table} created.")
# except Exception as e:
#     print(f"Table {project}.{bq_dataset}.{bq_table} already exists.")

# rows_to_insert = []

# if job.inspect_details and job.inspect_details.result:
#     result = job.inspect_details.result

#     processed_bytes = result.processed_bytes
#     total_estimated_bytes = result.total_estimated_bytes
#     info_type_stats = result.info_type_stats

#     for info_type_stat in info_type_stats:
#         info_type_name = info_type_stat.info_type.name
#         count = info_type_stat.count
#         rows_to_insert.append(
#             (processed_bytes, total_estimated_bytes, info_type_name, count)
#         )

# if rows_to_insert:
#     try:
#         errors = bigquery_client.insert_rows(table, rows_to_insert)
#         if not errors:
#             print(f"DLP findings have been stored in {project}.{bq_dataset}.{bq_table}.")
#         else:
#             print(f"Error inserting rows into BigQuery: {errors}")
#     except Exception as e:
#         print(f"Error inserting rows into BigQuery: {str(e)}")
# else:
#     print("No DLP findings to store in BigQuery.")

Inspection operation started: projects/axial-module-393809/locations/global/dlpJobs/i-9101630489374998451
DLP inspection completed.
name: "projects/axial-module-393809/locations/global/dlpJobs/i-9101630489374998451"
type_: INSPECT_JOB
state: DONE
inspect_details {
  requested_options {
    snapshot_inspect_template {
    }
    job_config {
      storage_config {
        big_query_options {
          table_reference {
            project_id: "axial-module-393809"
            dataset_id: "axial_dataset_sample"
            table_id: "sample_data"
          }
        }
      }
      inspect_config {
        info_types {
          name: "FIRST_NAME"
        }
        info_types {
          name: "LAST_NAME"
        }
        info_types {
          name: "EMAIL_ADDRESS"
        }
        info_types {
          name: "PASSWORD"
        }
        min_likelihood: LIKELY
        limits {
        }
      }
      actions {
        save_findings {
          output_config {
            table {
     